# Data Wrangling

This is one of the hardest parts of working with data. Raw data is rarely ready to be analyzed. Scientists spend a lot of time manipulating their data to get it into a form that can be used. We call it "wrangling".

Possible problems with data:
* Getting the data you need 
* Data Types
* Incorrect units

# Polar Vortex (January 2019) [CBS News Report](https://www.cbsnews.com/news/polar-vortex-2019-record-cold-grips-the-midwest-as-chicago-river-turns-to-ice/)


During the polar vortex, record low temperatures were set around Chicago. On Northerly Island, the National Weather Service recorded a low. One of the nodes (001e06113cf1) is on Michigan Avenue and Randolph Street by Millenium Park, not far from Northerly Island.

## How cold did it get during the polar vortex?

Data for the node **001e06113cf1** has already been extracted into dataset `polarvortex.csv` for you to use.

## Filtering the data you need

AoT nodes collect all sorts of data. For this question, we just want the temperature data. So we will have to filter out the data we don't need.

In [1]:
import pandas as pd

In [2]:
polar = pd.read_csv('data/polarvortex.csv')
polar.head()

,Unnamed: 0,timestamp,node_id,subsystem,sensor,parameter,value_raw,value_hrf
0,4097665,2019-01-30 00:00:24,001e06113cf1,lightsense,apds_9006_020,intensity,65535,5267.409
1,4097666,2019-01-30 00:00:24,001e06113cf1,lightsense,hih6130,humidity,65535,100.0
2,4097667,2019-01-30 00:00:24,001e06113cf1,lightsense,hih6130,temperature,65535,125.01
3,4097668,2019-01-30 00:00:24,001e06113cf1,lightsense,hmc5883l,magnetic_field_x,-1,-0.909
4,4097669,2019-01-30 00:00:24,001e06113cf1,lightsense,hmc5883l,magnetic_field_y,-1,-0.909


Let's first figure out what measurements the node is recording.

In [3]:
polar['parameter'].drop_duplicates()

0            intensity
1             humidity
2          temperature
3     magnetic_field_x
4     magnetic_field_y
5     magnetic_field_z
11            pressure
16                  id
17      acceleration_x
18      acceleration_y
19      acceleration_z
Name: parameter, dtype: object

Here is the key step where we filter and create a new dataframe that only contains the temperature readings.

In [4]:
temp = polar[(polar['parameter'] == "temperature")]

In [5]:
temp

,Unnamed: 0,timestamp,node_id,subsystem,sensor,parameter,value_raw,value_hrf
2,4097667,2019-01-30 00:00:24,001e06113cf1,lightsense,hih6130,temperature,65535,125.01
8,4097673,2019-01-30 00:00:24,001e06113cf1,lightsense,tmp421,temperature,65535,-0.06
12,4097677,2019-01-30 00:00:24,001e06113cf1,metsense,bmp180,temperature,65535,241.0
15,4097680,2019-01-30 00:00:24,001e06113cf1,metsense,htu21d,temperature,65535,128.86
20,4097685,2019-01-30 00:00:24,001e06113cf1,metsense,pr103j2,temperature,396,-15.4
...,...,...,...,...,...,...,...,...
169012,4266677,2019-01-31 23:59:57,001e06113cf1,metsense,bmp180,temperature,65535,241.0
169015,4266680,2019-01-31 23:59:57,001e06113cf1,metsense,htu21d,temperature,65535,128.86
169020,4266685,2019-01-31 23:59:57,001e06113cf1,metsense,pr103j2,temperature,378,-16.7
169022,4266687,2019-01-31 23:59:57,001e06113cf1,metsense,tmp112,temperature,65535,-254.0


To find the low temperature during the polar vortex, we can use the `.min()` method.

In [6]:
temp['value_hrf'].min()

'-0.06'

This doesn't make sense. We can see that there are lower values than -0.06. The hint is in the quotes.

Scientific data, like most data, is messy. Unfortunately, there is so much of it, we can't look at it line by line to search for bad data. Fortunately, there are some good techniques we can use to sniff out bad data.

A computer sees the values differently than we do. Don't assume.

In [7]:
temp.dtypes

Unnamed: 0     int64
timestamp     object
node_id       object
subsystem     object
sensor        object
parameter     object
value_raw     object
value_hrf     object
dtype: object

`object` means string. The `.min()` method only works for numbers.

In [8]:
temp['value_hrf'] = pd.to_numeric(temp['value_hrf'], errors='coerce')

<ipython-input-8-c71cd71a77d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['value_hrf'] = pd.to_numeric(temp['value_hrf'], errors='coerce')


In [9]:
temp.dtypes

Unnamed: 0      int64
timestamp      object
node_id        object
subsystem      object
sensor         object
parameter      object
value_raw      object
value_hrf     float64
dtype: object

# Working with Values

Now that we have our temperatures in `float`, we can do some math on them. The temperatures for all the sensors are in Celsius. Let's convert them to Fahrenheit so they are easier to understand.

$$ T(F) = \frac{9}{5}T(C)+32 $$

In [10]:
def Celsius_to_Fahrenheit(temp_C):
    temp_F = (temp_C * 9/5) + 32
    return temp_F

In [11]:
temp['Temp_F'] = temp['value_hrf'].apply(Celsius_to_Fahrenheit)

temp.head()

<ipython-input-11-9b13c2c6f8a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['Temp_F'] = temp['value_hrf'].apply(Celsius_to_Fahrenheit)


,Unnamed: 0,timestamp,node_id,subsystem,sensor,parameter,value_raw,value_hrf,Temp_F
2,4097667,2019-01-30 00:00:24,001e06113cf1,lightsense,hih6130,temperature,65535,125.01,257.018
8,4097673,2019-01-30 00:00:24,001e06113cf1,lightsense,tmp421,temperature,65535,-0.06,31.892
12,4097677,2019-01-30 00:00:24,001e06113cf1,metsense,bmp180,temperature,65535,241.00,465.800
15,4097680,2019-01-30 00:00:24,001e06113cf1,metsense,htu21d,temperature,65535,128.86,263.948
20,4097685,2019-01-30 00:00:24,001e06113cf1,metsense,pr103j2,temperature,396,-15.40,4.280


# -----------------------Lectue Stops Here---------------------------

# Begin Small Group Exercise

### Let's get an idea of the range of temperature values.


In [12]:
temp['Temp_F'].min()

-425.2

In [13]:
temp['Temp_F'].describe()

count    47327.000000
mean       143.656079
std        284.182250
min       -425.200000
25%         -1.660000
50%        257.018000
75%        418.550000
max        465.800000
Name: Temp_F, dtype: float64

There is a large variation in the temperatures (value_hrf) for the different sensors (-425&deg;F to 466&deg;F).

In [14]:
temp['sensor'].drop_duplicates()

2     hih6130
8      tmp421
12     bmp180
15     htu21d
20    pr103j2
22     tmp112
24     tsys01
Name: sensor, dtype: object

It looks like we have multiple temperature sensors. 

### Which temperature sensor is the most accurate?

We can narrow our search by getting an idea of the temperature range for each sensor. Even though these values sit in the same column, we can use the `groupby` method to analyze each sensor individually.

In [15]:
sensor_range = temp.groupby(['sensor'])
sensor_range['Temp_F'].describe()

,count,mean,std,min,25%,50%,75%,max
sensor,,,,,,,,
bmp180,6761.0,465.80000,4.036181e-11,465.800,465.800,465.800,465.800,465.800
hih6130,6761.0,257.01800,2.035145e-11,257.018,257.018,257.018,257.018,257.018
htu21d,6761.0,263.94800,2.688893e-11,263.948,263.948,263.948,263.948,263.948
pr103j2,6761.0,-6.41545,5.528077e+00,-16.150,-10.120,-8.320,-1.660,6.440
tmp112,6761.0,-425.20000,2.643414e-11,-425.200,-425.200,-425.200,-425.200,-425.200
tmp421,6761.0,31.89200,8.136316e-13,31.892,31.892,31.892,31.892,31.892
tsys01,6761.0,418.55000,2.387600e-11,418.550,418.550,418.550,418.550,418.550


It looks like `pr103j2` might be the best candidate because its minimum is closest to the National Weather Services record low (-22&deg;C).